In [1]:
import numpy as np
import pandas as pd
import math
import datetime
import time
import os
import glob
import multiprocessing
import tqdm

In [2]:
%run py/constants.py
%run py/take_processing_methods.py
%run py/marker_processing_methods.py
%run py/heatmap_methods.py
%run py/grasp_property_methods.py
%run py/rasterization_methods.py

In [3]:
def get_transformed_marker(frame_index, marker_id, df, rot_matrices):
    rot_matrix = rot_matrices[df.index.get_loc(frame_index)].T

    rb_pivot =[df[df.columns[6]].ix[frame_index], df[df.columns[7]].ix[frame_index], df[df.columns[8]].ix[frame_index]]

    marker = [df[marker_id + "_X"].ix[frame_index], df[marker_id + "_Y"].ix[frame_index], df[marker_id + "_Z"].ix[frame_index]]  
    marker = inv_trans(marker, rb_pivot, rot_matrix)

    return marker

def quaternion_matrix(row, rigid_body_name):
    """
    Converts a quaternion to a rotation matrix.
    Source: http://fabiensanglard.net/doom3_documentation/37726-293748.pdf
    RETURN: Rotation matrix as numpy array.
    """
    x = row[rigid_body_name + '_X_Rotation']
    y = row[rigid_body_name + '_Y_Rotation']
    z = row[rigid_body_name + '_Z_Rotation']
    w = row[rigid_body_name + '_W_Rotation']
    
    rot_matrix = [
        1.0-2*y*y-2*z*z, 2.0*x*y+2*w*z, 2.0*x*z - 2*w*y,
        2.0*x*y - 2*w*z, 1.0-2*x*x-2*z*z, 2.0*y*z+2*w*x,
        2.0*x*z+2*w*y, 2.0*y*z-2*w*x, 1.0-2*x*x-2*y*y]
    
    return rot_matrix


def inv_trans(p, pos, rot):
    """
    Transform a marker into the rigid body coordinate system
    """
    p = np.subtract(p, pos)
    p = np.dot(p, rot.T)
    p = [p[0], p[2], p[1]]
    return p

def trans(prow):
    markers = ["Wrist", "R_Shape_1","R_Shape_2","R_Shape_3","R_Shape_4",
               "Thumb_Fn", "Thumb_DIP", "Thumb_PIP" ,"Thumb_MCP", "Index_Fn", "Index_DIP", "Index_PIP","Index_MCP", 
               "Middle_Fn", "Middle_DIP", "Middle_PIP", "Middle_MCP", "Ring_Fn", "Ring_DIP", "Ring_PIP","Ring_MCP", 
               "Little_Fn", "Little_DIP", "Little_PIP", "Little_MCP"]
    row = prow.copy(deep=True)
    
    # iterate over all rows
    for m in markers:
        
        x = row[m + "_X"]
        y = row[m + "_Y"]
        z = row[m + "_Z"]
        
        pivot = row['Pivot']
        rot_matrix = row['RotationMatrix']
        
        if not math.isnan(x):
            marker = [x, y, z]
            marker = inv_trans(marker, pivot, rot_matrix)
            
            row[m + "_X"] = marker[0]
            row[m + "_Y"] = marker[1]
            row[m + "_Z"] = marker[2]
            
    return row

def log(s):
    with open("./out/status_PY01_preprocessing.txt", "a") as myfile:
        myfile.write("[" + str(datetime.datetime.now()) + "] " + s + "\n")
    #print("[" + str(datetime.datetime.now()) + "] " + s)


test_participants = [1,2,9,15,19]

def doJob(filename):
    if filename.endswith(".csv"):
        current_participant = filename.split(".")[0]
        cp = int(current_participant.replace("P","").split("_")[0])
        if cp not in test_participants:
            rigid_body_name = filename.split("_")[1]

            if os.path.isfile("Pickles/dfTransformed_" + current_participant + ".pkl") :
                log(current_participant + " is already available. Skipped.")
            else:
                log("Reading " + current_participant)
                df, t_adjusted = read_take("raw-data/" + filename)

                df.Time = df.Time + t_adjusted
                df.Time = df.Time * 1000 # sec to ms
                df.Time = df.Time.astype(np.int64)
                # Cells for quaternions
                x = df[df.columns[2]]
                y = df[df.columns[3]]
                z = df[df.columns[4]]
                w = df[df.columns[5]]

                rot_matrix = np.array([
                    [1-2*y*y-2*z*z, 2*x*y+2*w*z, 2*x*z - 2*w*y],
                    [2*x*y - 2*w*z, 1-2*x*x-2*z*z, 2*y*z+2*w*x],
                    [2*x*z+2*w*y, 2*y*z-2*w*x, 1-2*x*x-2*y*y]])

                angle = np.degrees(np.arccos(np.dot((rot_matrix[:,1,:].T), [0, 1, 0])))
                #adf = pd.DataFrame(angle)
                #df2 = adf[np.logical_not(adf[0].isnull())]
                #df3 = df2[df2[0] < df2[0].mean() + df2[0].std()*2]
                dfFinal = df#.iloc[df3.index]

                log("Creating RotationMatrix")
                dfFinal['RotationMatrix'] = dfFinal.apply(lambda row : quaternion_matrix(row, rigid_body_name), axis=1)
                dfFinal.RotationMatrix = dfFinal.RotationMatrix.apply(lambda x : np.array(x).reshape(3,3))
                dfFinal['Pivot'] = dfFinal.apply(lambda row : [row[rigid_body_name + '_X'], row[rigid_body_name + '_Y'], row[rigid_body_name + '_Z']], axis=1)
                dfFinal['Pivot'] = dfFinal.Pivot.apply(lambda x : np.array(x))

                log("Applying transformation")
                dfTransformed = dfFinal.apply(lambda x : trans(x), axis=1)
                log("Saving to File")
                dfTransformed.to_pickle("Pickles/dfTransformed_" + current_participant + ".pkl")        

In [4]:
%%time
print("HINT: This may take some hours. Detailed progress see output file: out/status_PY01_preprocessing.txt")
pool = multiprocessing.Pool(max(multiprocessing.cpu_count()-1, 1))
files = list(os.listdir("raw-data/"))
for _ in tqdm.tqdm(pool.imap_unordered(doJob, files), total=len(files)):
    pass

HINT: This may take some hours. Detailed progress see output file: out/status_PY01_preprocessing.txt


100%|██████████| 130/130 [25:10<00:00, 11.62s/it] 

CPU times: user 5.4 s, sys: 5.61 s, total: 11 s
Wall time: 25min 11s
